In [1]:
import math
from math import sqrt, exp, log  # exp(n) == e^n, log(n) == ln(n)
import scipy.optimize as so
import numpy as np
import yfinance as yf
import pandas as pd
import functions as fn
from datetime import date

In [2]:
def compute_log_likelihood(params, *args):
    '''
    Compute the average Log Likelihood, this function will by minimized by scipy.
    Find in (2.2) in linked paper

    returns: the average log likelihood from given parameters
    '''
    # functions passed into scipy's minimize() needs accept one parameter, a tuple of
    #   of values that we adjust to minimize the value we return.
    #   optionally, *args can be passed, which are values we don't change, but still want
    #   to use in our function (e.g. the measured heights in our sample or the value Pi)

    theta, mu, sigma = params
    X, dt = args
    n = len(X)

    sigma_tilde_squared = sigma ** 2 * (1 - exp(-2 * mu * dt)) / (2 * mu)
    summation_term = 0

    for i in range(1, len(X)):
        summation_term += (X[i] - X[i - 1] * exp(-mu * dt) - theta * (1 - exp(-mu * dt))) ** 2

    summation_term = -summation_term / (2 * n * sigma_tilde_squared)

    log_likelihood = (-log(2 * math.pi) / 2) + (-log(sqrt(sigma_tilde_squared))) + summation_term

    return -log_likelihood
    # since we want to maximize this total log likelihood, we need to minimize the
    #   negation of the this value (scipy doesn't support maximize)


def estimate_coefficients_MLE(X, dt, tol=1e-10):
    '''
    Estimates Ornstein-Uhlenbeck coefficients (θ, µ, σ) of the given array
    using the Maximum Likelihood Estimation method

    input: X - array-like time series data to be fit as an OU process
           dt - time increment (1 / days(start date - end date))
           tol - tolerance for determination (smaller tolerance means higher precision)
    returns: θ, µ, σ, Average Log Likelihood
    '''

    bounds = ((None, None), (1e-5, None), (1e-5, None))  # theta ∈ ℝ, mu > 0, sigma > 0
                                                           # we need 1e-10 b/c scipy bounds are inclusive of 0, 
                                                           # and sigma = 0 causes division by 0 error
    theta_init = np.mean(X)
    initial_guess = (theta_init, 100, 100)  # initial guesses for theta, mu, sigma
    result = so.minimize(compute_log_likelihood, initial_guess, args=(X, dt), bounds=bounds, tol=tol)
    theta, mu, sigma = result.x 
    max_log_likelihood = -result.fun  # undo negation from __compute_log_likelihood
    # .x gets the optimized parameters, .fun gets the optimized value
    return theta, mu, sigma, max_log_likelihood

In [3]:
def compute_portfolio_values(ts_A, nameA, ts_B, namesB, slopes, beta):
    '''
    Compute the portfolio values over time when holding $1 of stock A 
    and -$alloc_B of stock B
    
    input: ts_A - time-series of price data of stock A,
           ts_B - time-series of price data of stock B
    outputs: Portfolio values of holding $1 of stock A and -$alloc_B of stock B
    '''
    
    ts_A = ts_A.copy()  # defensive programming
    ts_B = ts_B.copy()
    ts_A[nameA] = ts_A[nameA] / ts_A[nameA].iloc[0]
    prices = slopes[namesB].multiply(ts_B[namesB])
    asset_price = prices.sum(1)
    base = slopes[namesB].iloc[0].multiply(ts_B[namesB].iloc[0])
    ts_B["syntheticAsset"] = asset_price / base.sum()
    return ts_A[nameA] - beta * ts_B["syntheticAsset"]

In [4]:
def arg_max_B_alloc(ts_A, nameA, ts_B, namesB, slopes, dt):
    '''
    Finds the $ allocation ratio to stock B to maximize the log likelihood
    from the fit of portfolio values to an OU process

    input: ts_A - time-series of price data of stock A,
           ts_B - time-series of price data of stock B
           dt - time increment (1 / days(start date - end date))
    returns: θ*, µ*, σ*, B*
    '''
    
    theta = mu = sigma = alloc_B = 0
    max_log_likelihood = 0

    def compute_coefficients(beta):
        portfolio_values = compute_portfolio_values(ts_A, nameA, ts_B, namesB, slopes, beta)
        print("SINGLE TS:", portfolio_values)
        coeffs = estimate_coefficients_MLE(portfolio_values, dt)
        print(coeffs)
        return coeffs
    vectorized = np.vectorize(compute_coefficients)
    linspace = [1]#np.linspace(.01, 1, 100)
    res = vectorized(linspace)
    print(res)
    index = res[3].argmax()
    
    return res[0][index], res[1][index], res[2][index], linspace[index]

In [5]:
backtest_data = pd.read_csv("../backtest_data_entries_exits_2018.csv")
#Final price of each day
#backtest_data["datetime"] = pd.to_datetime(backtest_data["datetime"])
#backtest_data["dayPeriod"] = backtest_data["datetime"].dt.to_period("D")
backtest_daily = backtest_data#.groupby(backtest_data["dayPeriod"]).apply(lambda x: x.iloc[[-1]])


slopes = pd.read_csv("../hedge_ratios_2018.csv")
slopes["datetime"] = backtest_data["datetime"]
#slopes["dayPeriod"] = slopes["datetime"].dt.to_period("D")
slopes_daily = slopes#.groupby(slopes["dayPeriod"]).apply(lambda x: x.iloc[[-1]])

print(backtest_daily.shape, slopes_daily.shape)

(88327, 12) (88327, 10)


In [6]:
print(slopes_daily.columns)
slopes_daily.rename(columns={"hasHR":"hasclose", "ttwoHR":"ttwoclose", "ctasHR":"ctasclose", "sbuxHR":"sbuxclose", "alxnHR":"alxnclose"}, inplace=True)
print(slopes_daily.columns)

Index(['Unnamed: 0', 'hasHR', 'aaplHR', 'ttwoHR', 'sbuxHR', 'ctasHR', 'alxnHR',
       'algnHR', 'payxHR', 'datetime'],
      dtype='object')
Index(['Unnamed: 0', 'hasclose', 'aaplHR', 'ttwoclose', 'sbuxclose',
       'ctasclose', 'alxnclose', 'algnHR', 'payxHR', 'datetime'],
      dtype='object')


In [7]:
qqq = backtest_daily[["datetime", "qqqclose"]]
synth = backtest_daily[["datetime", "hasclose", "ttwoclose", "sbuxclose", "ctasclose", "alxnclose"]]
start = date(2018, 1, 2)
end = date(2018, 12, 31)
dt = 1.0 / len(qqq)#(end-start).days

# get historical market data
#gld_price = gld.history(period="100d")
#slv_price = slv.history(period="100d")
#ts_gld = gld_price["Close"]
#ts_slv = slv_price["Close"]

In [8]:
theta, mu, sigma, b_alloc = arg_max_B_alloc(qqq, "qqqclose", synth, ["hasclose", "ttwoclose", "sbuxclose", "ctasclose", "alxnclose"], slopes_daily, dt) 

SINGLE TS: 0        0.000000
1        0.000969
2        0.000750
3        0.002219
4        0.002232
           ...   
88322   -0.083542
88323   -0.083064
88324   -0.082804
88325   -0.082737
88326   -0.082721
Length: 88327, dtype: float64
(-0.005107691029573813, 100.05213176208693, 0.24136160447824573, 5.697484233383085)
SINGLE TS: 0        0.000000
1        0.000969
2        0.000750
3        0.002219
4        0.002232
           ...   
88322   -0.083542
88323   -0.083064
88324   -0.082804
88325   -0.082737
88326   -0.082721
Length: 88327, dtype: float64
(-0.005107691029573813, 100.05213176208693, 0.24136160447824573, 5.697484233383085)
(array([-0.00510769]), array([100.05213176]), array([0.2413616]), array([5.69748423]))


In [9]:
print(theta, mu, sigma, b_alloc)

-0.005107691029573813 100.05213176208693 0.24136160447824573 1


In [10]:
from math import sqrt, exp
import scipy.integrate as si
import scipy.optimize as so
import numpy as np

def Prime(f, x, theta, mu, sigma, r, h=1e-5):
    # given f, estimates f'(x) using the difference quotient formula 
    # WARNING: LOWER h VALUES CAN LEAD TO WEIRD RESULTS
    return (f(x+h, theta, mu, sigma, r) - f(x, theta, mu, sigma, r)) / h 

def Prime2(f, x, theta, mu, sigma, r, c, h=1e-5):
    # given f, estimates f'(x) using the difference quotient formula 
    # WARNING: LOWER h VALUES CAN LEAD TO WEIRD RESULTS
    return (f(x+h, theta, mu, sigma, r, c) - f(x, theta, mu, sigma, r, c)) / h 

def F(x, theta, mu, sigma, r):
    # equation 3.3
    def integrand(u):
        return u**(r/mu - 1) * exp(sqrt(2*mu / sigma**2) * (x-theta)*u - u**2/2)
    return si.quad(integrand, 0, np.inf)[0]

def G(x, theta, mu, sigma, r):
    # equation 3.4
    def integrand(u):
        return u**(r/mu - 1) * exp(sqrt(2*mu / sigma**2) * (theta-x)*u - u**2/2)
    return si.quad(integrand, 0, np.inf)[0]

def b_star(theta, mu, sigma, r, c):
    # estimates b* using equation 4.3
    # def opt_func(b):
    #     # equation 4.3 in the paper with terms moved to one side
    #     return abs(F(b, theta, mu, sigma, r) - (b-c)*Prime(F, b, theta, mu, sigma, r))
    # bounds = ((.01, .99),)
    # result = so.minimize(opt_func, .5, bounds=bounds)

    b_space = np.linspace(0.1,0.9, 801)
    def func(b):
        return F(b, theta, mu, sigma, r) - (b-c)*Prime(F, b, theta, mu, sigma, r)
    
    return so.brentq(func, 0, 1)

def V(x, theta, mu, sigma, r, c):
    # OUR SELL SIGNAL
    # equation 4.2, solution of equation posed by 2.3
    
    b_star_val = b_star(theta, mu, sigma, r, c)
    
    if x < b_star_val:
        return (b_star_val - c) * F(x, theta, mu, sigma, r) / F(b_star_val, theta, mu, sigma, r)
    else:
        return x - c

def d_star(theta, mu, sigma, r, c):
    # estimates d* using equation 4.11
  
    def func(d):
        return (G(d, theta, mu, sigma, r) * (Prime2(V, d, theta, mu, sigma, r, c) - 1)) - (Prime(G, d, theta, mu, sigma, r) * (V(d, theta, mu, sigma, r, c) - d - c))

    # finds the root between the interval [0, 1]
    return so.brentq(func, 0, 1)

In [11]:
r = c = .05
b = b_star(theta, mu, sigma, r, c)
d = d_star(theta, mu, sigma, r, c)
b, d  # our optima

<ipython-input-10-6f78aa557b4a>:20: IntegrationWarning: The algorithm does not converge.  Roundoff error is detected
  in the extrapolation table.  It is assumed that the requested tolerance
  cannot be achieved, and that the returned result (if full_output = 1) is 
  the best which can be obtained.
  return si.quad(integrand, 0, np.inf)[0]


OverflowError: math range error

In [ ]:
def createPositions(prices, betas):
    dataWithPosition = pd.DataFrame()
    dfList = [group[1] for group in prices.groupby(prices['datetime'].dt.date)]
    betaList = [group[1] for group in betas.groupby(prices["datetime"].dt.date)]
    qqq_gain = 1
    synth_gain = 1
    
    for day, beta in dfList: #Iterate over single items in data
        print(day["qqqprice"])
        day['aboveOrBelowEMA'] = np.where(day['spread'] > day['ema'], 1, -1)
        day['signal'] = np.where(day['spread'] > day['upperband'], -1, np.nan)
        day['signal'] = np.where(day['spread'] < day['lowerband'], 1, day['signal'])
        day['signal'] = np.where(day['aboveOrBelowEMA'] * day['aboveOrBelowEMA'].shift(1) < 0, 
                                         0, day['signal'])
        day['signal'] = day['signal'].ffill().fillna(0)
        day['position'] = day['signal'].shift(1).fillna(0)
        if day['position'].iloc[-1] != 0:
            day['position'].iloc[-1] = 0
        dataWithPosition = dataWithPosition.append(day)      
    return dataWithPosition

In [ ]:
backtest_data = pd.read_csv("./data/backtest_data_entry_exit.csv")
slopes_data = pd.read_csv("./data/hedge_ratios.csv")
backtest_data["datetime"] = pd.to_datetime(backtest_data["datetime"])
slopes_data[]

createPositions(backtest_data, slopes_data)

In [ ]:
syntheticAssetLogPrice = data[['hasclose', 'ttwoclose', 'idxxclose', 'sbuxclose', 'ctasclose', 'alxnclose']].apply(np.log)
qqqLogPrice = np.log(data['qqqclose'].values)

In [ ]:
kf = fn.multivariateKalmanFilter(syntheticAssetLogPrice, qqqLogPrice)
state_means, state_covs = kf.filter(qqqLogPrice)
basket_size = len(syntheticAssetLogPrice.columns)
slopes = state_means[:, np.arange(0, basket_size, 1)]
intercept = state_means[:, basket_size]

In [ ]:
prices = backtest_data[['hasclose', 'aaplclose', 'ttwoclose','sbuxclose', 'ctasclose', 'alxnclose', 'algnclose', 'payxclose']].values
hedge_ratios = np.asarray([slopes.T[i][lookback - 1:] for i in range(len(slopes.T))]).T

In [ ]:
backtest_data['hedgeRatioHAS'] = slopes[:, 0][lookback - 1:]
backtest_data['hedgeRatioTTWO'] = slopes[:, 1][lookback - 1:]
backtest_data['hedgeRatioIDXX'] = slopes[:, 2][lookback - 1:]
backtest_data['hedgeRatioSBUX'] = slopes[:, 3][lookback -1 :]
backtest_data['hedgeRatioCTAS'] = slopes[:, 4][lookback - 1:]

In [ ]:
tradeLog, minuteDf = fn.constructTradeLog(backtest_data['datetime'].values, backtest_data['position'].values, 
                                backtest_data['qqqclose'].values, prices, hedge_ratios.round(3), lot_size = 1000, 
                               stoploss = None)
tradeLog.tail()

In [ ]:
returns_df = minuteDf[['datetime']]
returns_df['cumulative_returns'] = np.cumprod(1 + minuteDf['returns'])
returns_df = returns_df.set_index('datetime')
returns_df.plot(figsize=[15, 7])

In [ ]:
total_profit = np.cumsum(tradeLog['trade_profit'])
print('Trade Log cumulative profit was {:.3f}'.format(total_profit.iloc[-1]))

In [ ]:
cumulative_return = np.cumprod(1 + tradeLog['trade_returns']) - 1
print('Trade Log cumulative return was {:.3f}%'.format(cumulative_return.iloc[-1] * 100))

In [ ]:
minuteDf['datetime'] = pd.to_datetime(minuteDf['datetime'])
dailyReturns = fn.calculateDailyReturns(minuteDf[['datetime', 'returns']])
sharpeRatio = fn.calculateAnnualizedSharpeRatio(dailyReturns)
print('Annualized Sharpe Ratio: ', sharpeRatio)

In [ ]:
plt.figure(figsize=[15, 7])
plt.hist(tradeLog['trade_returns'], bins=75)
plt.axvline(tradeLog['trade_returns'].mean(), color='k', linestyle='dashed', linewidth=1)

In [ ]:
tradeLog['trade_returns'].quantile(0.1)

In [ ]:
plt.figure(figsize=[15, 7])
plt.hist(tradeLog['trade_profit'], bins=75)
plt.axvline(tradeLog['trade_profit'].mean(), color='k', linestyle='dashed', linewidth=1)

In [ ]:
plt.figure(figsize=[15, 7])
plt.hist(tradeLog['holdingPeriod'], bins=50)
plt.axvline(tradeLog['holdingPeriod'].mean(), color='k', linestyle='dashed', linewidth=1)
plt.axvline(tradeLog['holdingPeriod'].median(), color='r', linestyle='dashed', linewidth=1)

In [ ]:
tradeLog.loc[tradeLog['trade_returns'] <= -0.0025]